In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

import requests
import yfinance as yf

In [2]:
from sqlalchemy.dialects.postgresql import insert

def _postgres_do_nothing_on_conflict(table, conn, keys, data_iter):
    print(type(table.table))
    print(table.table)
    data = [dict(zip(keys, row)) for row in data_iter]
    insert_statement = insert(table.table).values(data)
    upsert_statement = insert_statement.on_conflict_do_nothing(
        constraint=f"{table.table.name}_pkey"
    )
    conn.execute(upsert_statement)

In [3]:
def get_all_tickers() -> str:
    r = requests.get(
        'https://quality.data.gov.tw/dq_download_json.php?nid=11549&md5_url=bb878d47ffbe7b83bfc1b41d0b24946e')
    return [i["證券代號"] for i in r.json()]

In [4]:
dsn = "postgresql+psycopg2://ttl:50770329@db:5432/yahoo"

In [5]:
engine = create_engine(dsn)

In [6]:
tickers = [ i+'.TW' for i in get_all_tickers() ]

In [7]:
columns = [ 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume' ]

In [8]:
stock = yf.download(
    tickers=tickers,
    start="2022-11-23",
    end="2022-11-24",
    interval="1d",
    ignore_tz=False
)

[*********************100%***********************]  1123 of 1123 completed

47 Failed downloads:
- 3698.TW: No data found for this date range, symbol may be delisted
- 020004.TW: No data found for this date range, symbol may be delisted
- 020008.TW: No data found for this date range, symbol may be delisted
- 00742.TW: No data found for this date range, symbol may be delisted
- 4141.TW: No data found for this date range, symbol may be delisted
- 020020.TW: No data found for this date range, symbol may be delisted
- 00743.TW: No data found for this date range, symbol may be delisted
- 020019.TW: No data found for this date range, symbol may be delisted
- 6289.TW: No data found for this date range, symbol may be delisted
- 9188.TW: No data found for this date range, symbol may be delisted
- 02001R.TW: No data found for this date range, symbol may be delisted
- 00691R.TW: No data found for this date range, symbol may be delisted
- 3383.TW: No data found for this date range, symbol may be d

In [9]:
stock.shape

(1, 6738)

In [10]:
stock.columns

MultiIndex([('Adj Close',   '0050.TW'),
            ('Adj Close',   '0051.TW'),
            ('Adj Close',   '0052.TW'),
            ('Adj Close',   '0053.TW'),
            ('Adj Close',   '0054.TW'),
            ('Adj Close',   '0055.TW'),
            ('Adj Close',   '0056.TW'),
            ('Adj Close',   '0057.TW'),
            ('Adj Close',   '0061.TW'),
            ('Adj Close', '006203.TW'),
            ...
            (   'Volume',   '9940.TW'),
            (   'Volume',   '9941.TW'),
            (   'Volume',  '9941A.TW'),
            (   'Volume',   '9942.TW'),
            (   'Volume',   '9943.TW'),
            (   'Volume',   '9944.TW'),
            (   'Volume',   '9945.TW'),
            (   'Volume',   '9946.TW'),
            (   'Volume',   '9955.TW'),
            (   'Volume',   '9958.TW')],
           length=6738)

In [11]:
stocks = []
for ticker in tickers:
    tmp_stock = stock[ [ (column, ticker) for column in columns] ].copy()
    tmp_stock.columns = columns
    tmp_stock['Ticker'] = ticker
    tmp_stock = tmp_stock[ ["Ticker"]+columns ]
    tmp_stock.reset_index(names='datetime', inplace=True)
    tmp_stock.dropna(subset=columns, how="all", axis=0, inplace=True)
    stocks += [ tmp_stock ]

In [12]:
stock = pd.concat(stocks)
stock.head()

,datetime,Ticker,Adj Close,Close,High,Low,Open,Volume
0,2022-11-23 00:00:00+08:00,0050.TW,115.599998,115.599998,115.900002,115.050003,115.650002,7938962.0
0,2022-11-23 00:00:00+08:00,0051.TW,51.700001,51.700001,51.799999,51.450001,51.650002,15130.0
0,2022-11-23 00:00:00+08:00,0052.TW,99.099998,99.099998,99.550003,98.750000,99.500000,466530.0
0,2022-11-23 00:00:00+08:00,0053.TW,55.250000,55.250000,55.250000,55.150002,55.200001,4175.0
0,2022-11-23 00:00:00+08:00,0055.TW,21.610001,21.610001,21.620001,21.320000,21.350000,353740.0


In [13]:
stock.columns = [ ''.join(x.lower().split()) for x in stock.columns.tolist() ]
stock.head()

,datetime,ticker,adjclose,close,high,low,open,volume
0,2022-11-23 00:00:00+08:00,0050.TW,115.599998,115.599998,115.900002,115.050003,115.650002,7938962.0
0,2022-11-23 00:00:00+08:00,0051.TW,51.700001,51.700001,51.799999,51.450001,51.650002,15130.0
0,2022-11-23 00:00:00+08:00,0052.TW,99.099998,99.099998,99.550003,98.750000,99.500000,466530.0
0,2022-11-23 00:00:00+08:00,0053.TW,55.250000,55.250000,55.250000,55.150002,55.200001,4175.0
0,2022-11-23 00:00:00+08:00,0055.TW,21.610001,21.610001,21.620001,21.320000,21.350000,353740.0


In [14]:
stock.to_sql(
    "yfinance",
    engine,
    if_exists="append",
    index=False,
    method=_postgres_do_nothing_on_conflict
)

OperationalError: (psycopg2.OperationalError) could not translate host name "db" to address: Temporary failure in name resolution

(Background on this error at: https://sqlalche.me/e/14/e3q8)